In [81]:
K = 5
D = 1

n = 2*K
d = 2*D

2

In [82]:
using NPZ
using IterTools

# Load the Fermi Dirac types
include("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/operators.jl")


# Load pretty print for operators
include("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/operator_term_enhanced.jl")

In [83]:
function multisets(D::Int, k::Int)::Vector{Vector{Int}}
    if k == 0
        return [Int[]]
    elseif D == 0
        return Vector{Vector{Int}}()
    else
        result = Vector{Vector{Int}}()
        for i in 0:k
            # choose i copies of 1,2,...,D-1 and then D last
            for rest in multisets(D - 1, k - i)
                push!(result, vcat(rest, fill(D, i)))  # append D's at the end
            end
        end
        return result
    end
end

function same_number_even_odd(xs)
    evens = count(x -> iseven(x), xs)
    odds  = count(x -> isodd(x), xs)
    return evens == odds
end

same_number_even_odd (generic function with 1 method)

In [84]:
X = [Op(a, true, true)*Op(i, false, true) + Op(a, true, false)*Op(i, false, false) for i in 1:K, a in 1:K]

5×5 Matrix{FD{Int64}}:
 a_{2}^†a_{2} + a_{1}^†a_{1}   …  a_{10}^†a_{2} + a_{9}^†a_{1}
 a_{2}^†a_{4} + a_{1}^†a_{3}      a_{10}^†a_{4} + a_{9}^†a_{3}
 a_{2}^†a_{6} + a_{1}^†a_{5}      a_{10}^†a_{6} + a_{9}^†a_{5}
 a_{2}^†a_{8} + a_{1}^†a_{7}      a_{10}^†a_{8} + a_{9}^†a_{7}
 a_{2}^†a_{10} + a_{1}^†a_{9}     a_{10}^†a_{10} + a_{9}^†a_{9}

In [85]:
basis = collect(subsets(1:n,d));

function base_lt(x, y, r)
    return length(x ∩ r) > length(y ∩ r)
end;

basis = filter(b -> same_number_even_odd(b) ,basis)

sort!(basis, lt=(x,y) -> base_lt(x,y, Set(1:d)));

In [87]:
h1es = []
eris = []
dist = []

h1e_names = readdir("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/Graphs/LiH$(D)e$(K)o/h1e"; join=true)
h1e_short_names = readdir("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/Graphs/LiH$(D)e$(K)o/h1e")

for (i,filename) in enumerate(h1e_names)
    if endswith(filename, ".npy")
        push!(h1es, npzread(filename))
        push!(dist, h1e_short_names[i][5:end-4])
    end
end

eri_names = readdir("Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/Graphs/LiH$(D)e$(K)o/eri"; join=true)

for filename in eri_names
    if endswith(filename, ".npy")
        push!(eris, npzread(filename))
    end
end

In [88]:
full_bas = Vector{Matrix{Int64}}([])

for k in 1:d
    for I in multisets(D, k)
        for B in vcat(collect(product(ntuple(i->D + 1:K, k)...))...)
            cont = false
            for r in 1:k - 1
                if I[r] == I[r + 1] && B[r] > B[r + 1]
                    cont = true
                end
            end

            for Bl in subsets(B, 3)
                if sort(Bl) == Bl
                    cont = true
                end
            end
            if cont
                continue 
            end;
            
            Tb = transpose(hcat([I,collect(B)]...))
            if !(Tb in full_bas)
                push!(full_bas, Tb)
            end
        end
    end
end

full_bas[11]

2×2 Matrix{Int64}:
 1  1
 2  5

In [89]:
fcb = with_limited_threads(1) do
    cb_local = [[1;zeros(length(basis) - 1, 1)][:,1]]
    
    for M in full_bas
        vec = matrixrep(prod([X[col[1],col[2]] for col in eachcol(M)]), basis, [basis[1]])[:,1]
        push!(cb_local, vec/sqrt(sum([v^2 for v in vec])))
    end
    
    cb_local
end

fcb = Matrix(hcat(fcb...))

Q, R = qr(fcb)
fcb = Matrix(Q);

In [90]:
hamiltonians = []

for (m,h1e) in enumerate(h1es)
    eri = eris[m]
    h1eO = sum([h1e[i,j]*X[i,j] for i in 1:K for j in 1:K]);

    eriO = sum([eri[i,j,k,l]*(Op(i,true, true)*Op(k,true, true)*Op(l, false, true)*Op(j,false, true) + Op(i,true, true)*Op(k,true, false)*Op(l, false, false)*Op(j,false, true) + Op(i,true, false)*Op(k,true, true)*Op(l, false, true)*Op(j,false, false) + Op(i,true, false)*Op(k,true, false)*Op(l, false, false)*Op(j,false, false))
        for i in 1:K for j in 1:K for k in 1:K for l in 1:K]);
    
    Ham = h1eO + (1/2)*eriO;

    hamil = with_limited_threads(1) do
        matrixrep(Ham, basis);
    end
    push!(hamiltonians, hamil)
end

In [91]:
# Create/recreate the eigen directory
eigen_dir = "Algebra-of-spin-restricted-CC/SpinRestrictedHomotopy/Graphs/LiH$(D)e$(K)o/eigen"
if isdir(eigen_dir)
    rm(eigen_dir, recursive=true)  # Delete existing directory and all contents
end
mkdir(eigen_dir)  # Create fresh directory

for (i,hamilton) in enumerate(hamiltonians)
    npzwrite(joinpath(eigen_dir, "Eigs_$(dist[i]).npy"), eigen(transpose(fcb)*hamilton*fcb).values)
end